# Analiza knjig


Analizirala bom 2400 najboljših knjig s seznama _nekaj_ spletne strani [GoodReads](https://www.goodreads.com/)

Sprva bomo naložili zajete podatke:

In [39]:
import pandas as pd
import os.path

pd.options.display.max_rows = 20

knjige = pd.read_csv(os.path.join('obdelani-podatki', 'knjige.csv'), index_col='id')
zanri = pd.read_csv(os.path.join('obdelani-podatki', 'zanri.csv'))

Sprva poglejmo podatke.

In [40]:
knjige

,avtor,naslov,ocena,glasovi,strani,leto,serija
id,,,,,,,
1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",4.32,6589817,374.0,2008,True
2,J.K. Rowling,Harry Potter and the Order of the Phoenix (Har...,4.50,2608155,870.0,2003,True
3,Harper Lee,To Kill a Mockingbird,4.28,4673257,324.0,1960,False
4,Jane Austen,Pride and Prejudice,4.27,3108362,279.0,1813,False
5,Stephenie Meyer,"Twilight (Twilight, #1)",3.61,5131640,501.0,2005,True
...,...,...,...,...,...,...,...
2394,Colleen Hoover,"Never Never (Never Never, #1)",4.06,72574,300.0,2015,True
2397,Brandon Mull,"Keys to the Demon Prison (Fablehaven, #5)",4.44,71900,593.0,2010,True
2397,Tana French,"The Likeness (Dublin Murder Squad, #2)",4.05,132533,466.0,2008,True


## 1. osnovni podatki

Najprej poglejmo, kakšno je povprečje glasov, strani in ocen, da bomo kasneje lažje ocenili, kako rezultati odstopajo od povprečja. Poglejmo tudi maksimalne in minimalne vrednosti.

In [10]:
round(knjige[['ocena', 'glasovi', 'strani']].mean(), 2)

ocena           4.07
glasovi    230770.65
strani        417.38
dtype: float64

In [11]:
knjige[['ocena', 'glasovi', 'strani']].max()

ocena            4.82
glasovi    7362544.00
strani        5216.00
dtype: float64

Največje število strani se nam zdi previsoko. Preverimo katere knjige imajo zelo visoko število strani.

In [44]:
knjige[(knjige.strani > 1600)]

,avtor,naslov,ocena,glasovi,strani,leto,serija
id,,,,,,,
9,J.R.R. Tolkien,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,4.60,113229,1728.0,1955,False
144,Laura Ingalls Wilder,"The Little House Collection (Little House, #1-9)",4.34,143786,2700.0,1971,True
286,Arthur Conan Doyle,The Complete Sherlock Holmes,4.47,179722,1796.0,1927,False
409,J.K. Rowling,"Harry Potter Series Box Set (Harry Potter, #1-7)",4.72,258358,4100.0,2007,True
480,Stephenie Meyer,"The Twilight Collection (Twilight, #1-3)",3.80,44105,1690.0,2007,True
921,Robert Musil,The Man Without Qualities,4.23,5760,1774.0,1930,False
1107,L.M. Montgomery,The Complete Anne of Green Gables Boxed Set (A...,4.44,101739,2088.0,1908,True
1133,E.L. James,"Fifty Shades Trilogy (Fifty Shades, #1-3)",4.11,115162,1859.0,2005,True
1319,Christopher Paolini,"Eragon, Eldest &amp; Brisingr (Inheritance, #1-3)",4.23,32537,1952.0,2008,True


Število strani je tako visoko, ker je v enem naslovu združena cela serija knjig. Teh knjig v prihodnje ne bomo obravnavali, saj imamo knjige iz serij zabeležene tudi posamično.

In [12]:
knjige[['ocena', 'glasovi', 'strani']].min()

ocena       3.11
glasovi    38.00
strani      0.00
dtype: float64

Zmoti nas, da imajo nekatere knjige 0 strani. To se je zgodilo, ker nekatere knjige na spletni strani nimajo zabeleženih strani knjige. Preverimo, koliko je takih knjig.

In [19]:
knjige[(knjige.strani == 0)]

,avtor,naslov,ocena,glasovi,strani,leto,serija
id,,,,,,,
1503,Ayn Rand,Atlas Shrugged &amp; The Fountainhead,4.1,4575,0.0,1995,False


Sedaj dodatno prefiltriramo podatke.

In [54]:
knjige = knjige[(knjige.strani > 0) & (knjige.strani < 1600)]

Sedaj se lotimo prvega vprašanja. Kateri avtor je najbolj priljubljen. Preverili bomo torej, kateri avtor se je na seznamu pojavil največkrat. Predpostavimo, da je to Stephen King, saj ima dolgoletno kariero, širok opus, veliko njegovih del pa je bilo adaptiranih za film, kar je njegova dela spoznalo še s širšim občinstvom.

In [59]:
po_avtorjih = knjige.groupby('avtor')
po_avtorjih.size().sort_values(ascending=False).head(5)

avtor
Stephen King           34
Terry Pratchett        22
Rick Riordan           19
Nicholas Sparks        17
William Shakespeare    16
dtype: int64

Rezultat ni presenetljiv. Vsi avotrji so napisali ogromno knjig, ki so bile v veliko primerih upodobljene v film.

## 2. Analiza gledo na leto